In [ ]:
import json, time, os
from functools import reduce

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm.autonotebook import tqdm

from py2neo import Graph, Node, Relationship

In [ ]:
public_address = '18.27.79.39'
graph = Graph('bolt://{}:7687'.format(public_address), auth=('neo4j','myneo'))

def run_query(query, graph, print_query=False, run_query=True, 
              print_only=False, to_df=False, verbose=True):
    df = 1
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        if to_df:
            df = graph.run(query).to_data_frame()
        else:
            graph.run(query)
    end_time = time.time()
    minutes_elapsed = (end_time-start_time)/60
    if verbose:
        print("Query completed in {:.2f} minutes.".format(minutes_elapsed))
    return df

In [ ]:
# Write edge lists for quanta that cite other quanta
min_year, max_year = 1900, 2020
def write_edge_list_citing_quanta(min_year, max_year):
    for y in tqdm(range(min_year, max_year+1)):
        print("{}...".format(y))

        # Write edge list for graph at year=y
        query = """
        CALL apoc.export.csv.query('
        MATCH (z:Year)<-[:PUBLISHED_IN]-(a:Quanta)-[:CITES]->(b:Quanta)-[:PUBLISHED_IN]->(y:Year)
        WHERE z.year<={year} AND y.year<={year}
        RETURN id(a) as source, id(b) as target
        ','{out}',
        {{quotes:false}});
        """.format(year=y, out='/import/embeddings/all_quanta/quanta.{}.edgelist'.format(y))
        run_query(query, graph, print_only=False)
        
write_edge_list_citing_quanta(min_year, max_year)

In [ ]:
# Running on ipython tmux pane 1
# Write edge lists for author_quanta
min_year, max_year = 1900, 2020
def write_edge_list_author_quanta(min_year, max_year):
    for y in tqdm(range(min_year, max_year+1)):
        print("{}...".format(y))

        # Write edge list for graph at year=y
        query = """
        CALL apoc.export.csv.query('
        MATCH (z:Year)<-[:PUBLISHED_IN]-(q:Quanta)<-[:AUTHORED]-(a:Author)
        WHERE z.year<={year}
        RETURN id(a) as source, id(q) as target
        ','{out}',
        {{quotes:false}});
        """.format(year=y, out='/import/embeddings/author_quanta/quanta.{}.edgelist'.format(y))
        run_query(query, graph, print_only=False)
    
write_edge_list_author_quanta(min_year, max_year)

In [ ]:
# Running on ipython tmux pane 2
# Write edge lists for venue_quanta
min_year, max_year = 1900, 2020
def write_edge_list_venue_quanta(min_year, max_year):
    for y in tqdm(range(min_year, max_year+1)):
        print("{}...".format(y))

        # Write edge list for graph at year=y
        query = """
        CALL apoc.export.csv.query('
        MATCH (z:Year)<-[:PUBLISHED_IN]-(q:Quanta)-[:PUBLISHED_IN]->(v:Venue)
        WHERE z.year<={year}
        RETURN id(q) as source, id(v) as target
        ','{out}',
        {{quotes:false}});
        """.format(year=y, out='/import/embeddings/venue_quanta/quanta.{}.edgelist'.format(y))
        run_query(query, graph, print_only=False)
    
write_edge_list_venue_quanta(min_year, max_year)

In [ ]:
# Replace commas with spaces
find . -name 'quanta.*.edgelist' -exec sed -i "s/,/ /g" {} \;

In [ ]:
for filename in quanta.*.edgelist; do
     tail -n +2 "$filename" > "$filename.tmp"
     mv -f "$filename.tmp" "$filename" 
done

In [ ]:
# Run node2vec
for filename in quanta.*.edgelist; do
     ./node2vec -i:"$filename" -o:"$filename.emb" -d:12 -l:80 -dr -v
done

In [ ]:
# Import results to graph
min_year, max_year = 1900, 2020
for y in tqdm(range(min_year, max_year+1)):
    print("{}...".format(y))
    
    edgelist = '/import/embeddings/quanta.{}.edgelist'.format(y)
    
    # Write edge list for graph at year=y
    query = """
    CALL apoc.load.csv('/import/embeddings/quanta.{year}.edgelist.emb', 
        {{sep:" ", header:true}})
    YIELD list 
    WITH apoc.convert.toInteger(head(list)) as nodeId, 
        [x IN tail(list) | apoc.convert.toFloat(x)] AS embedding
    MATCH (n) WHERE id(n)=nodeId
    MATCH (y:Year) WHERE y.year={year}
    MERGE(n)-[r:METRICS_IN]->(y)
    SET r.node2vec=embedding
    """.format(year=y)
    run_query(query, graph, print_only=False)